In [21]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [22]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('pubmed','splits/pubmed_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [23]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "pubmed\pubmed10.pt"
GNNpubmed=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.6).to(device)
GNNpubmed.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [24]:
#prendo l'embedding dalla rete
GNNpubmed.eval()
out = GNNpubmed.emb(list_mat, layer_norm)
#torch.save(out,'pubmed\emb_pubmed25.pt')


In [25]:
out.shape

torch.Size([19717, 448])

In [26]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

,source,target,weight
0,0,1378,1
1,0,1544,1
2,0,6092,1
3,0,7636,1
4,0,14442,1
...,...,...,...
44322,19076,19559,1
44323,19093,19223,1
44324,19125,19369,1
44325,19198,19266,1


In [27]:
#definizione distanza manhattan
import numpy as np

def manhattan_distance(arr1, arr2):
    """
    Calculates the Manhattan distance between two arrays of the same length.
    """
    return np.sum(np.abs(arr1 - arr2))

In [28]:
'''#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('pubmed\pubmed_weights.csv', index = False)'''

"#aggiungo i pesi ottenuti dall'embedding\ndf_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])\nadj_mat['weight'] = adj_mat.apply(lambda x:\n                                    manhattan_distance(df_emb.loc[x['target']].values,\n                                    df_emb.loc[x['source']].values),\n                                    axis=1)\nadj_mat.to_csv('pubmed\\pubmed_weights.csv', index = False)"

In [29]:
#aggiungo i pesi ottenuti dall'embedding
from sklearn.metrics.pairwise import cosine_similarity

df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    cosine_similarity(np.array([df_emb.iloc[x['target']]]),
                                    np.array([df_emb.iloc[x['source']]]))[0][0],
                                    axis=1)
adj_mat.to_csv('pubmed/pubmed_cosine_weights10.csv', index = False)

In [ ]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('pubmed\pubmed_labels.csv', index = False)